## How vocational students who have studied ICT are placed after one and three years from graduation

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt 

ImportError: DLL load failed while importing _multiarray_umath: Määritettyä osaa ei löydy.

ImportError: numpy._core.multiarray failed to import

### Load Raw Data

This data was taken from Tilastokeskus in .csv - format:
https://pxdata.stat.fi:443/PxWeb/sq/69e3b920-5e97-4e39-9fdc-61bbfbcfdfa9 

To install sklearn and pandas, open miniconda CMD and run:
```cmd
    pip install pandas
    pip install sklearn
    pip install matplotlib
```


NOTE: The data has a title-row, which has to be skipped (skiprows), also the encoding needs to be 'ISO-8850-1' (default UTF-8 doesn't work) and delimiter needs to be given also.

To see how the data is read into the variable, it helps to install Data Wrangler - extension. It can be used to view the data in a table format.

In [ ]:
# load data
if os.path.exists('./csv/tivi_2020_2023.csv'):
   df =  pd.read_csv('./csv/tivi_2020_2023.csv', encoding='ISO-8859-1', delimiter=';', engine='python', skiprows=2)

NameError: name 'os' is not defined

The data has information about all the vocational ICT students who graduatated one year before the statistical year (2020, 2021, 2022, 2023), how the students have been placed after one year of graduation: employed, employed fulltime, employed students, students fulltime, unemployed, others. The same information is found also for three years after graduation. 
Data is gathered by regions (Ahvenanmaa, Varsinaissuomi, Satakunta, Kanta-Häme, Pirkanmaa, Päijät-Häme, Kymenlaakso, Etelä-Karjala, Etelä-Savo, Pohjois-Karjala, Pohjois-Savo, Keskisuomi, Etelä-Pohjanmaa, Pohjanmaa,Keski-Pohjanmaa, Pohjois-Pohjanmaa, Kainuu, Lappi). Data is categorized also by gender. Each value is presented as count as well as percentage.

It seems that there are no rows missing data.

In [ ]:
df.head()

We will consentrate only on these columns (in %): Vuosi, Asuinmaakunta, Päätoimiset työlliset vuosi valmistumisen jälkeen (%), Työlliset opiskelijat vuosi valmistumisen jälkeen (%), Päätoimiset opiskelijat vuosi valmistumisen jälkeen (%), Työttömät vuosi valmistumisen jälkeen (%) ja Muut vuosi valmistumisen jälkeen (%)

In [ ]:
df_selected = df[['Vuosi', 'Asuinmaakunta', 'Päätoimiset työlliset vuosi valmistumisen jälkeen (%)', 'Työlliset opiskelijat vuosi valmistumisen jälkeen (%)', 'Päätoimiset opiskelijat vuosi valmistumisen jälkeen (%)', 'Työttömät vuosi valmistumisen jälkeen (%)', 'Muut vuosi valmistumisen jälkeen (%)']].copy()
df_selected.dropna() # drops rows with null values
df_selected.info()

The data, which has tekstual values e.g. labels needs to be converted into numbers so that machine learning algorithm can use them.

[Label encoding](https://vitalflux.com/labelencoder-example-single-multiple-columns/#:~:text=Label%20encoding%20technique%20is%20implemented%20using%20sklearn%20LabelEncoder.,single%20and%20multiple%20columns%20in%20Python%20Pandas%20Dataframe.)

The regions (asuinmaakunta) labels should be converted into numbers.

In [ ]:
df_selected.head()

In [ ]:
le=preprocessing.LabelEncoder()
arr=le.fit_transform(df_selected['Asuinmaakunta'])
df_selected['Asuinmaakunta']=arr
print(arr)
df_selected.head()

### Linear regression

In [ ]:
lr = linear_model.LinearRegression()
y=df_selected['Työttömät vuosi valmistumisen jälkeen (%)']
X=df_selected.drop(columns='Työttömät vuosi valmistumisen jälkeen (%)')
fig, ax = plt.subplots(figsize=(10,20))
ax.plot([y.min(), y.max()],[y.min(),y.max()], 'k--', lw=4)
